In [2]:
import Pkg; Pkg.add("RobotZoo"); Pkg.add("StaticArrays")
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots; plotly()
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
# using TrajOptPlots
# using StaticArrays
using Printf
using RobotZoo
using StaticArrays

   Resolving package versions...
  No Changes to `C:\Users\ologa\Documents\CMU\Semester2\16.745-OptimalControlReinforcementLearning\Optimal-Controls-Gymnastics-Bot\MonkeyBarBot\Project.toml`
  No Changes to `C:\Users\ologa\Documents\CMU\Semester2\16.745-OptimalControlReinforcementLearning\Optimal-Controls-Gymnastics-Bot\MonkeyBarBot\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\ologa\Documents\CMU\Semester2\16.745-OptimalControlReinforcementLearning\Optimal-Controls-Gymnastics-Bot\MonkeyBarBot\Project.toml`
  No Changes to `C:\Users\ologa\Documents\CMU\Semester2\16.745-OptimalControlReinforcementLearning\Optimal-Controls-Gymnastics-Bot\MonkeyBarBot\Manifest.toml`
  Activating environment at `C:\Users\ologa\Documents\CMU\Semester2\16.745-OptimalControlReinforcementLearning\Optimal-Controls-Gymnastics-Bot\MonkeyBarBot\Project.toml`


In [3]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils", "monkeyBotDynamics.jl"))

monkeybot_dynamics (generic function with 1 method)

In [5]:
#Copied from RobotZoo.acrobot() src
function monkeybot_dynamics(params::NamedTuple, x::Vector, u::Vector)
    g = 9.81
    m1,m2 = params.m1, params.m2
    l1,l2 = params.l1, params.l2
    J1,J2 = params.I1, params.I2
    θ1,    θ2    = x[1], x[2]
    θ1dot, θ2dot = x[3], x[4]
    s1,c1 = sincos(θ1)
    s2,c2 = sincos(θ2)
    c12 = cos(θ1 + θ2)

    # mass matrix
    m11 = m1*l1^2 + J1 + m2*(l1^2 + l2^2 + 2*l1*l2*c2) + J2
    m12 = m2*(l2^2 + l1*l2*c2 + J2)
    m22 = l2^2*m2 + J2
    M = [m11 m12; m12 m22]

    # bias term
    tmp = l1*l2*m2*s2
    b1 = -(2 * θ1dot * θ2dot + θ2dot^2)*tmp
    b2 = tmp * θ1dot^2
    B = [b1, b2]

    # friction
#     c = 1.0
#     C = [c*θ1dot, c*θ2dot]

    # gravity term
    g1 = ((m1 + m2)*l2*c1 + m2*l2*c12) * g
    g2 = m2*l2*c12*g
    G = [g1, g2]
    # equations of motion
    τ = [0, u[1]]
    θddot = M\(τ - B - G)
    return [θ1dot, θ2dot, θddot[1], θddot[2]]
end

monkeybot_dynamics (generic function with 2 methods)

In [6]:
#Stance Dynamics
function stance1_dynamics(params::NamedTuple, x::Vector, u::Vector)
    #Hand 1 in contact with bar
    ẋ = monkeybot_dynamics(params, x, u)
    return ẋ
end


#Jump maps
function jump1_map(x)
    #Hand 1 experiences inelastic collision with bar
    
    #Zero out hand 1 linear velocity
    #Constraint is that transform to hand 1 velocity equals 0 and hand 1 position equals bar position
    θ1, θ2 = x[1], x[2]
#     deltaX = BarCoordsGoal[1] - BarCoordsBase[1]
#     θ1_star = acos((deltaX - l1*cos(θ1))/l2)
    θ1_star = -(pi - θ1 - θ2)
    θ2_star = -θ2
    
    return [θ1_star, θ2_star, 0, 0]
end


#Generate Integrator Function
function dynamics_rk4(params::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(params, x,        u)
    k2 = dt * ode(params, x + k1/2, u)
    k3 = dt * ode(params, x + k2/2, u)
    k4 = dt * ode(params, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end


dynamics_rk4 (generic function with 1 method)

In [7]:
#Cost Functions
function cost(params::NamedTuple, Z::Vector)::Real
    #Calculate cost over whole trajectory
    N, xic, idx = params.N, params.xic, params.idx
    Q, R, Qf = params.Q, params.R, params.Qf
    
    cost = 0.0
    for k = 1:(N-1)
        ##Maybe try using Xref for shoulder trajectory as cost to make sure it stays on that path
            ## Might interfere with free swing at maximum swing
        xk = Z[idx.x[k]]
        uk = Z[idx.u[k]]
        
        if (k == 1)
            xk_tilde = xk - xic
            cost += 0.5*(xk_tilde'*Qf*xk_tilde + uk'*R*uk)
        else
            cost += 0.5*(xk'*Q*xk + uk'*R*uk)
        end
    end
    xN = Z[idx.x[N]]
    cost += 0.5*(xN'*Qf*xN)
    
    return cost
end

function cost_minTime(params::NamedTuple, Z::Vector)::Real
    #Calculate cost over whole trajectory
    N, idx = params.N, params.idx
    Q, R, Qf = params.Q, params.R, params.Qf
    
    cost = 0.0
    for k = 1:(N-1)
        ##Maybe try using Xref for shoulder trajectory as cost to make sure it stays on that path
            ## Might interfere with free swing at maximum swing
        xk = Z[idx.x[k]]
        ui = Z[idx.u[k]]
        uk = ui[1]
        hk = ui[2]
        
        if (k == 1)
            xk_tilde = xk - xic
            cost += 0.5*hk*(xk_tilde'*Qf*xk_tilde + uk'*R*uk)
        else
            cost += 0.5*hk*(xk'*Q*xk + uk'*R*uk)
        end
    end
    xN = Z[idx.x[N]]
    cost += 0.5*(xN'*Qf*xN)
    
    return cost
end


cost_minTime (generic function with 1 method)

In [8]:
# #Generate initial trajectory
# function generateInitialTrajectory(params::NamedTuple, Z::Vector)
# end

# function generateSimpleTrajectory(params::NamedTuple, Z::Vector)
#     #Generate initial simplified version of center body trajectory (sinusoidal path)
    
# end

# initialTrajectory = generateInitialTrajectory(params, x, u)
# plot(initialTrajectory)

In [9]:
function create_idx(nx,nu,N) #copied from homeworks
    # create idx for indexing convenience
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    # h_i = Z[idx.h[i]]
    # and stacked dynamics constraints of size nx are 
    # c[idx.c[i]] = <dynamics constraint at time step i>
    #
    # feel free to use/not use this 
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
#     return (nx=nx, nu=nu, N=N, nz=nz, nc=nc, x=x, u=u, h=h, c = c)
    return (nx=nx, nu=nu, N=N, nz=nz, nc=nc, x=x, u=u, c = c)
end

function dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, J1 = params.M1, params.J1
    
    c = zeros(eltype(Z), idx.nc)
    
    # TODO: input walker dynamics constraints (constraints 3-6 in the opti problem)
    for k = 1:(N-1)
        xk = Z[idx.x[k]]
        uk = Z[idx.u[k]]
        if !(k in J1) #Stance 1 dynamics
            xk1 = dynamics_rk4(params, monkeybot_dynamics, xk, uk, dt)
            c[idx.c[k]] = Z[idx.x[k+1]] - xk1 #Add dynamics constraint on state
        else
            xk1 = jump1_map(dynamics_rk4(params, monkeybot_dynamics, xk, uk, dt))
            c[idx.c[k]] = Z[idx.x[k+1]] - xk1 #Add dynamics constraint on state
        end
        xk1 = dynamics_rk4(params, monkeybot_dynamics, xk, uk, dt)
        c[idx.c[k]] = Z[idx.x[k+1]] - xk1 #Add dynamics constraint on state
    end
    
    return c 
end

function dynamics_constraints_minTime(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M1, J1 = params.M1, params.J1
    
    c = zeros(eltype(Z), idx.nc)
    
    # TODO: input walker dynamics constraints (constraints 3-6 in the opti problem)
    for k = 1:(N-1)
        xk = Z[idx.x[k]]
        ui = Z[idx.u[k]]
        uk = [ui[1]]
        hk = ui[2]
        if !(k in J1) #Stance 1 dynamics
            xk1 = dynamics_rk4(params, monkeybot_dynamics, xk, uk, hk)
            c[idx.c[k]] = Z[idx.x[k+1]] - xk1 #Add dynamics constraint on state
        else
            xk1 = jump1_map(dynamics_rk4(params, monkeybot_dynamics, xk, uk, hk))
            c[idx.c[k]] = Z[idx.x[k+1]] - xk1 #Add dynamics constraint on state
        end
    end
    
    return c 
end

function equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    l1, l2 = params.l1, params.l2
    BarCoords = params.BarCoords
    J1 = params.J1
    
    #Stack up all equality constraints
    initialCondition = Z[idx.x[1]] - xic
#     initialConditions = zeros(eltype(Z), 2)
#     θ1k, θ2k = Z[idx.x[1]]
#     xk = BarCoords[1][1] + l1*sin(θ1k) + l2*sin(θ1k + θ2k)
#     yk = BarCoords[1][2] - (l1+l2) - (l1*sin(θ1k) + l2*sin(θ1k + θ2k))
#     initialConditions[1] = BarCoords[1][1] - xk
#     initialConditions[2] = BarCoords[1][2] - yk
    
    terminalCondition = Z[idx.x[1]] - xg
#     terminalConditions = zeros(eltype(Z), length(J1)*2)
    
#     barIdx = 1
#     for k = 1:length(J1)
#         θ1k, θ2k = Z[idx.x[k]]
#         barCoords = BarCoords[barIdx+1]
#         xk = barCoords[1] + l1*sin(θ1k) + l2*sin(θ1k + θ2k)
#         yk = barCoords[2] - (l1*sin(θ1k) + l2*sin(θ1k + θ2k))
#         terminalConditions[2*k-1] = xk
#         terminalConditions[2*k] = yk
#         barIdx += 1
#     end
    dynamicsConstraints = dynamics_constraints(params, Z)
    
    return [initialCondition; terminalCondition; dynamicsConstraints]
end

function equality_constraint_minTime(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    l1, l2 = params.l1, params.l2
    BarCoords = params.BarCoords
    J1 = params.J1
    
    #Stack up all equality constraints
    initialCondition = Z[idx.x[1]] - xic
    initialBarConditions = zeros(eltype(Z), 2)
    θ1k, θ2k = Z[idx.x[1]]
    xk = BarCoords[1][1] + l1*sin(θ1k) + l2*sin(θ1k + θ2k)
    yk = BarCoords[1][2] - (l1+l2) - (l1*sin(θ1k) + l2*sin(θ1k + θ2k))
    initialBarConditions[1] = BarCoords[1][1] - xk
    initialBarConditions[2] = BarCoords[1][2] - yk
    
    terminalCondition = Z[idx.x[N]] - xg
    barConditions = zeros(eltype(Z), length(J1)*2)
    barIdx = 1
    for k = 1:length(J1)
        θ1k, θ2k = Z[idx.x[k]]
        barCoords = BarCoords[barIdx+1]
        xk = barCoords[1] + l1*sin(θ1k) + l2*sin(θ1k + θ2k)
        yk = barCoords[2] - (l1*cos(θ1k) + l2*cos(θ1k + θ2k))
        barConditions[2*k-1] = xk
        barConditions[2*k] = yk
        barIdx += 1
    end
    dynamicsConstraints = dynamics_constraints_minTime(params, Z)
    
    return [initialCondition; terminalCondition; dynamicsConstraints]
end

function inequality_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
        
    #Write inequality constraints for the problem
#     c = zeros(eltype(Z), 3*N
#     for k = 1:(N-1)
# #         uk = Z[idx.u[k]]
# #         c[3*k] = uk[2]
#         c[k] = xk[1]
#     end
    return []
end


inequality_constraint (generic function with 1 method)

In [10]:
θ2_guess_half = [0, 1.570796e-01, 3.141593e-01, 3.141593e-01, 3.141593e-01, 1.570796e-01,
            0, -1.570796e-01, -3.141593e-01, -4.712389e-01, -6.283185e-01, -7.853982e-01, 
            -9.424778e-01, -7.853982e-01, -6.283185e-01, -4.712389e-01, -3.141593e-01, 
            -1.570796e-01, 0, 1.570796e-01, 3.141593e-01, 1.570796e-01, 0, -1.570796e-01, 
            -3.141593e-01, -4.712389e-01, -6.283185e-01, -3.141593e-01, 0, 3.141593e-01, 
            6.283185e-01, 7.853982e-01, 9.424778e-01, 1.193989e+00, 1.445500e+00]
θ1_guess_half = [0, 0, 0, 7.853982e-02, 1.570796e-01, 2.356194e-01, 3.141593e-01, 
            3.141593e-01, 3.141593e-01, 2.356194e-01, 1.570796e-01, 0, -1.570796e-01, 
            -2.356194e-01, -3.141593e-01, -3.926991e-01, -4.712389e-01, -5.497787e-01, 
            -6.283185e-01, -6.283185e-01, -6.283185e-01, -5.497787e-01, -4.712389e-01, 
            -3.926991e-01, -3.141593e-01, -2.356194e-01, -1.570796e-01, -7.853982e-02, 
            0, 7.853982e-02, 1.570796e-01, 3.141593e-01, 4.712389e-01, 6.596694e-01, 8.481000e-01]



# θ2_guess = vcat(θ2_guess_half,reverse(θ2_guess_half))
# θ1_guess = vcat(θ1_guess_half,reverse(θ1_guess_half))

θ2_guess = θ2_guess_half
θ1_guess = θ1_guess_half

35-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.07853982
  0.1570796
  0.2356194
  0.3141593
  0.3141593
  0.3141593
  0.2356194
  0.1570796
  0.0
 -0.1570796
  ⋮
 -0.3926991
 -0.3141593
 -0.2356194
 -0.1570796
 -0.07853982
  0.0
  0.07853982
  0.1570796
  0.3141593
  0.4712389
  0.6596694
  0.8481

In [11]:
# problem size 
nx = 4
nu = 1
nu_minTime = 2
# B_1 = [1,0,3,0,5] #Bar indecies for hand 1
# B_2 = [0,2,0,4,0] #Bar indecies for hand 2
# B_1 = [1,2,3,4,5]
# B_N = [20,10,10,10,10]
B_1 = [1]
B_N = [length(θ1_guess)]
@show B_N

dt = 0.05
tf = dt*(sum(B_N)-1)
t_vec = 0:dt:tf
N = length(t_vec)
Nbars = 5
g = 9.81
l1 = 1
l2 = 1
m1 = 1
m2 = 1
I1 = 0.2
I2 = 0.2

BarCoords = [zeros(2) for i = 1:Nbars]
for i = 1:Nbars
    barX = (i-1)*1.5
    barY = 0
    BarCoords[i] = [barX; barY]
end
@show BarCoords

#Inital and final conditions
xic = [0,0,0,0] #Start hanging
@show θ1g = pi/2-(pi-acos((l1^2 + l2^2 - 1.5^2)/2*l1*l2))/2
@show θ2g = pi-acos((l1^2 + l2^2 - 1.5^2)/2*l1*l2)
xg = [θ1g,θ2g,0,0]
@show xg+
# index sets 
B1 = vcat([B_1[i] for i = 1:length(B_1) for j = 1:B_N[i]])
M1 = [i for i = 1:length(B1)]
J1 = [sum(B_N[1:i]) for i = 1:length(B_N)]
    
# end
@show B1
# @show B2
@show M1
# @show M2
@show J1
# @show J2

# # Create reference trajectory for shoulder joint
# Xref, Uref = shoulder_reference_trajectory(params, xic, xg, dt, N)

# LQR cost function
Q = diagm([0.01; 0.01; 0.01; 0.01]);
R = 1000
Qf = 10*Q;

# create indexing utilities 
idx = create_idx(nx,nu_minTime,N)
@show N
# Params
params = (
    nx = nx,
    nu = nu_minTime,
    tf = tf, 
    dt = dt, 
    t_vec = t_vec,
    N = N, 
    M1 = M1, 
    J1 = J1, 
    B1 = B1, 
    BarCoords = BarCoords,
    xic = xic, 
    xg = xg,
    idx = idx,
    Q = Q, R = R, Qf = Qf,
    g = g,
    l1 = l1, l2 = l2,
    I1 = I1, I2 = I2,
    m1 = m1, m2 = m2,
)

#Primal bounds
x_l = zeros(idx.nz)
x_u =  zeros(idx.nz)
θ1_bound = (θ1g + pi/8)
θ2_bound = (θ2g + pi/8)
for i = 1:N
    x_l[idx.x[i]] = [-θ1_bound, -θ2_bound, -Inf, -Inf]
    x_u[idx.x[i]] = [θ1_bound, θ2_bound, Inf, Inf]
    if (i < N)
        x_l[idx.u[i]] = [-Inf, 0.05]
        x_u[idx.u[i]] = [Inf, 0.2]
    end
end

#Inequality constraint bounds
# c_l = -(l1+l2-1 + 0.5)*ones(N)
# c_u = (l1+l2-1 - 0.5)*ones(N)
# c_l = [-(pi-(θ2g-pi/8)) for i = 1:(N-1)]
# c_u = [(pi-(θ2g-pi/8)) for i = 1:(N-1)]
c_l = []
c_u = []
# @show size(c_l)


B_N = [35]
BarCoords = [[0.0, 0.0], [1.5, 0.0], [3.0, 0.0], [4.5, 0.0], [6.0, 0.0]]
θ1g = pi / 2 - (pi - acos((((l1 ^ 2 + l2 ^ 2) - 1.5 ^ 2) / 2) * l1 * l2)) / 2 = 0.848062078981481
θ2g = pi - acos((((l1 ^ 2 + l2 ^ 2) - 1.5 ^ 2) / 2) * l1 * l2) = 1.445468495626831
xg = [0.848062078981481, 1.445468495626831, 0.0, 0.0]
B1 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
M1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
J1 = [35]
N = 35


Any[]

In [12]:

#Initialize z0 with reference shoulder trajectory
z0 = zeros(idx.nz) # update this

# θ2_guess = [sin(i*0.5) for i in t_vec]

# θ1_guess = [cos(i*0.5) for i in t_vec]


for i = 1:(N-1)
    θ1dot_i = (θ1_guess[i+1] - θ1_guess[i])/dt
    θ2dot_i = (θ2_guess[i+1] - θ2_guess[i])/dt
    z0[idx.x[i]] = [θ1_guess[i], θ2_guess[i], θ1dot_i, θ2dot_i]
    z0[idx.u[i]] = [0, dt]
end
z0[idx.x[N]] = [θ1_guess[N], θ2_guess[N], 0, 0]

# adding a little noise to the initial guess is a good idea 
z0 = z0 + (1e-6)*randn(idx.nz)

#Choose diff type
diff_type = :auto 

#Solve trajectory
Z = fmincon(cost,equality_constraint,inequality_constraint,
            x_l,x_u,c_l,c_u,z0,params,diff_type;

#             tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = true)
# Z = fmincon(cost_minTime,equality_constraint_minTime,inequality_constraint,
#             x_l,x_u,c_l,c_u,z0,params,diff_type;

            tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = false)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



208-element Vector{Float64}:
    0.8480620786169304
    1.4454684945876528
    6.85102205210009e-16
    2.483329799371599e-16
 -194.54069350548082
    0.054415856377146035
    0.9761253258571483
    1.0976756590991004
    5.773722520639281
  -15.230851730557324
   65.71973919384936
    0.054415856377146035
    1.233596367243242
    ⋮
 -150.74017550608923
    0.054415856377146035
   -1.2407611487757488
   -0.9250043580048034
    1.8955716424896603
  -18.433395349004147
 -109.65337249286785
    0.054415856377146035
   -1.2407611685601603
   -1.838167572717321
   -2.7973641698737644
  -17.499570798899192

In [13]:
# pull the X and U solutions out of Z 
X = [Z[idx.x[i]] for i = 1:N]
U = [Z[idx.u[i]] for i = 1:(N-1)]

# X = [z0[idx.x[i]] for i = 1:N]


#Post process to get x and y of hands
Xm = zeros(4,N)
x1 = zeros(N)
y1 = zeros(N)
x2 = zeros(N)
y2 = zeros(N)
for i = 1:N
    xB = BarCoords[1][1]
    yB = BarCoords[1][2]
    θ1, θ2, θ1dot, θ2dot = X[i]
    x1[i] = xB + l1*sin(θ1)
    y1[i] = yB - l1*cos(θ1)
    x2[i] = x1[i] + l2*sin(θ1 + θ2)
    y2[i] = y1[i] - l2*cos(θ1 + θ2)
end
x1_check, y1_check = (BarCoords[1][1] + l1*sin(xic[1])), (BarCoords[1][2] - l1*cos(xic[1]))
x2_check, y2_check = (x1_check + l2*sin(xic[1] + xic[2])), (y1_check - l2*cos(xic[1] + xic[2]))
# @show x1N_check, y1N_check = (BarCoords[1][1] + l1*sin(xg[1])), (BarCoords[1][2] - l1*cos(xg[1]))
# @show x2N_check, y2N_check = (x1N_check + l2*sin(xg[1] + xg[2])), (y1N_check - l2*cos(xg[1] + xg[2]))
# @show X[N][1:2] - xg[1:2]
# # @test norm([x1[1], y1[1]] - [x1_check, y1_check]) <= 1e-6
# # @test norm([x2[1], y2[1]] - [x2_check, y2_check]) <= 1e-6
# # @test norm([x1[N], y1[N]] - [x1N_check, y1N_check]) <= 1e-6
# # @test norm([x2[N], y1[N]] - [x2N_check, y2N_check]) <= 1e-6


# #Plot joint positions
# h = scatter([BarCoords[i][1] for i = 1:5], [BarCoords[i][2] for i = 1:5], marker = :circle, seriescolor = :blue, label = "bar",
# aspect_ratio = :equal, xlims = (-2,8), ylims = (-4,4))

# a = @animate for i ∈ 1:length(x1)-1
#     plot!([tuple(x1[i:i+1], y1[i:i+1])], marker = :circle, seriescolor = :red, legend = false)
#     plot!([tuple(x2[i:i+1], y2[i:i+1])], marker = :circle, seriescolor = :green, legend = false)
# #     Plots.frame(a, b)
# end
# gif(a, fps = 4)

#Plot joint angles
plot([0; [sum([U[j][2] for j = 1:i]) for i = 1:(N-1)]], [X[i][1] for i = 1:N], label = "Theta 1")
plot!([0; [sum([U[j][2] for j = 1:i]) for i = 1:(N-1)]], [X[i][2] for i = 1:N], label = "Theta 2")


In [14]:
include(joinpath(@__DIR__, "utils","walker.jl"))
model = (g = 9.81, mb= 0.0, mf = 1.0, ℓ_min = 0.5, ℓ_max = 1.5,l1=l1,l2 = l2)
vis = Visualizer()
dt = params.dt
build_walker!(vis, model::NamedTuple)
anim = mc.Animation(floor(Int,tf/dt))

for i = 1:N
    xB = BarCoords[1][1]
    yB = BarCoords[1][2]
    θ1, θ2, θ1dot, θ2dot = X[i]
#     θ1 = θ1_guess[i]
#     θ2 = θ2_guess[i]
    state_i = [θ1,θ2,xB,yB]
    contact_mode = 1
    mc.atframe(anim, i) do
        update_monkey_pose!(vis,contact_mode,model::NamedTuple,params::NamedTuple, state_i)
    end
end
mc.setanimation!(vis, anim)
display(render(vis))

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
└ http://127.0.0.1:8700


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("Rfoot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x52, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "torso" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("body" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x62, 0x6f, 0x64, 0x79], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Laxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x4c, 0x61, 0x78, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Lleg" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x4c, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Rleg" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x52, 0x6c, 0x65, 0x67, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Raxle" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x73, 0x6f, 0x2f, 0x52, 0x61, 0x78, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Lfoot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x4c, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x4c, 0x66, 0x6f, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))))), Set{Any}(), ip"127.0.0.1", 8700))